# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san pedro pochutla,15.7333,-96.4667,24.28,82,30,1.20,MX,1736220041
1,1,ribeira grande,38.5167,-28.7000,18.76,93,98,14.31,PT,1736219775
2,2,nema,57.5067,50.5011,-21.97,100,62,1.67,RU,1736219996
3,3,guerrero negro,27.9769,-114.0611,15.70,58,4,4.18,MX,1736220045
4,4,shimoda,34.6667,138.9500,10.37,65,2,13.44,JP,1736219886


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    size = "Humidity",
    color="City",
    alpha = 0.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) &
                           (city_data_df["Wind Speed"] < 4.5) &
                           (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,cabo san lucas,22.8909,-109.9124,21.53,61,0,2.24,MX,1736220006
75,75,hawaiian paradise park,19.5933,-154.9731,26.04,77,0,4.12,US,1736219779
148,148,nova sintra,14.8667,-24.7167,23.14,66,0,4.26,CV,1736220212
311,311,tura,25.5198,90.2201,21.70,52,0,0.40,IN,1736220406
342,342,vicuna,-30.0319,-70.7081,21.85,52,0,0.45,CL,1736220442
401,401,dabakala,8.3667,-4.4333,23.72,26,0,0.51,CI,1736220512
410,410,boa vista,2.8197,-60.6733,26.99,61,0,4.12,BR,1736220299
451,451,macapa,0.0389,-51.0664,23.99,94,0,1.54,BR,1736220484
507,507,jonuta,18.0833,-92.1333,21.14,96,0,1.41,MX,1736220638
513,513,ixtapa,20.7000,-105.2000,21.79,88,0,1.54,MX,1736220645


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,cabo san lucas,MX,22.8909,-109.9124,61,
75,hawaiian paradise park,US,19.5933,-154.9731,77,
148,nova sintra,CV,14.8667,-24.7167,66,
311,tura,IN,25.5198,90.2201,52,
342,vicuna,CL,-30.0319,-70.7081,52,
401,dabakala,CI,8.3667,-4.4333,26,
410,boa vista,BR,2.8197,-60.6733,61,
451,macapa,BR,0.0389,-51.0664,94,
507,jonuta,MX,18.0833,-92.1333,96,
513,ixtapa,MX,20.7000,-105.2000,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
      "categories": "accommodation.hotel",
      "apiKey": geoapify_key,
      "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
hawaiian paradise park - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
tura - nearest hotel: Hotel Polo Orchid
vicuna - nearest hotel: No hotel found
dabakala - nearest hotel: Hôtel Résidence les villages
boa vista - nearest hotel: Aipana Plaza Hotel
macapa - nearest hotel: Equatorial Hotel
jonuta - nearest hotel: No hotel found
ixtapa - nearest hotel: Ma. Cristina
narasannapeta - nearest hotel: No hotel found
bucerias - nearest hotel: Grand Decameron Complex, A Trademark All Inclusive


,City,Country,Lat,Lng,Humidity,Hotel Name
21,cabo san lucas,MX,22.8909,-109.9124,61,Comfort Rooms
75,hawaiian paradise park,US,19.5933,-154.9731,77,No hotel found
148,nova sintra,CV,14.8667,-24.7167,66,Residência Ka Dencho
311,tura,IN,25.5198,90.2201,52,Hotel Polo Orchid
342,vicuna,CL,-30.0319,-70.7081,52,No hotel found
401,dabakala,CI,8.3667,-4.4333,26,Hôtel Résidence les villages
410,boa vista,BR,2.8197,-60.6733,61,Aipana Plaza Hotel
451,macapa,BR,0.0389,-51.0664,94,Equatorial Hotel
507,jonuta,MX,18.0833,-92.1333,96,No hotel found
513,ixtapa,MX,20.7000,-105.2000,88,Ma. Cristina


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    size = "Humidity",
    color="City",
    alpha = 0.75,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)